UTS, MDSI, Autumn 2021 , Big Data Engineering(94693)

Assignment 3 - Streaming with Kafka and Spark Streaming, 13 June 2021

Group Project by
* Irfan
* Ming 
* Jedo
* Hnin Pwint Tin / 13738339  

AT3_Part2.ipynb

The SQL script in this file are to run in KSQL DB editor.
Each block of cell is to create Stream, Table to answer the business queries.

Prior to running this script, AT3_Producer.py has to be run.

## KSQL - Part 2

#### Create Table from the Kafka Topic - "nsw_car_park_facility"

In [ ]:
### Create TABLE - Car Park Facility
CREATE TABLE car_park_facility_table
(facility_id VARCHAR PRIMARY KEY, 
 facility_name VARCHAR,
 park_id VARCHAR,
 time VARCHAR,
 spots int,
 message_date VARCHAR,
 tfnsw_facility_id VARCHAR,
 facility_occupancy_total int, 
)
WITH (KAFKA_TOPIC='nsw_car_park_facility', VALUE_FORMAT='JSON');

#### Create Stream from the Kafka Topic - "nsw_car_park_zone"

In [ ]:
### Create STREAM - Car Park Facility -- Working
CREATE STREAM car_park_zone_stream
(zone_id VARCHAR,
facility_id VARCHAR,
zone_name VARCHAR,
spots int,
parent_zone_id VARCHAR,
message_date VARCHAR,
zone_occupancy_total int)
WITH (KAFKA_TOPIC='nsw_car_park_zone', VALUE_FORMAT='JSON');

## Business Queries

#### Car Park Facility Occupancy Percentage

In [ ]:
CREATE TABLE SEVEN_HILLS_OCCUPANCY_RATE AS
SELECT
  F.FACILITY_ID FACILITY_ID,
  F.FACILITY_NAME FACILITY_NAME,
  F.message_date MESSAGE_DATE,
  ((F.FACILITY_OCCUPANCY_TOTAL * 100) / F.SPOTS) OCCUPANCY_PERCENT
FROM CAR_PARK_FACILITY_TABLE F
WHERE F.FACILITY_NAME = 'Seven hills'
EMIT CHANGES;

In [ ]:
CREATE STREAM SEVEN_HILLS_ZONE_OCCUPANCY_RATE AS   
	SELECT
		z.zone_id as zone_id ,
		z.zone_name as zone_name,
        z.facility_id as facility_id,
		fa.facility_name as facility_name,
        z.zone_occupancy_total as zone_occupancy_total,
        z.spots as spots,
        ((Cast(z.zone_occupancy_total as Double) / Cast(z.spots as Double))* 100)  as occupancy_rate,
        z.message_date as time
	FROM CAR_PARK_ZONE_STREAM as z
	LEFT JOIN CAR_PARK_FACILITY_TABLE as fa ON z.facility_id = fa.facility_id
	WHERE fa.FACILITY_NAME = 'Seven hills'
EMIT CHANGES;

In [ ]:
CREATE TABLE KELLYVILLE_OCCUPANCY_RATE AS
SELECT
  F.FACILITY_ID FACILITY_ID,
  F.FACILITY_NAME FACILITY_NAME,
  F.message_date MESSAGE_DATE,
  ((F.FACILITY_OCCUPANCY_TOTAL * 100) / F.SPOTS) OCCUPANCY_PERCENT
FROM CAR_PARK_FACILITY_TABLE F
WHERE F.FACILITY_NAME = 'Kellyville Station Car Park'
EMIT CHANGES;

In [ ]:
CREATE STREAM KELLYVILLE_ZONE_OCCUPANCY_RATE AS   
	SELECT
		z.zone_id as zone_id ,
		z.zone_name as zone_name,
        z.facility_id as facility_id,
		fa.facility_name as facility_name,
        z.zone_occupancy_total as zone_occupancy_total,
        z.spots as spots,
        ((Cast(z.zone_occupancy_total as Double) / Cast(z.spots as Double))* 100)  as occupancy_rate,
        z.message_date as time
	FROM CAR_PARK_ZONE_STREAM as z
	LEFT JOIN CAR_PARK_FACILITY_TABLE as fa ON z.facility_id = fa.facility_id
	WHERE fa.FACILITY_NAME = 'Kellyville Station Car Park'
EMIT CHANGES;

#### Car Park Facilities Occupied More Than 90 Percent

In [ ]:
## Working / Runnin with Success
CREATE TABLE CAR_PARK_FACILITY_OCCUPANCY_RATE_GREATER_THAN_90 AS 
  SELECT
    f.facility_id,
    f.facility_name,
    f.message_date,
    f.facility_occupancy_total,
    f.spots,
	((Cast(f.facility_occupancy_total as Double) / Cast(f.spots as Double))* 100) as occupancy_rate
  FROM    as f
  WHERE ((Cast(f.facility_occupancy_total as Double) / Cast(f.spots as Double))* 100) > 90
EMIT CHANGES;

#### Occupancy Rate of Car Park Zones

In [ ]:
## Join Facility and Zone
CREATE STREAM BELLA_FACILITY_ZONE_OCCUPANCY_RATE AS   
	SELECT
		z.zone_id as zone_id ,
		z.zone_name as zone_name,
        z.facility_id as facility_id,
		fa.facility_name as facility_name,
        z.zone_occupancy_total as zone_occupancy_total,
        z.spots as spots,
        ((Cast(z.zone_occupancy_total as Double) / Cast(z.spots as Double))* 100)  as occupancy_rate,
        z.message_date as time
	FROM CAR_PARK_ZONE_STREAM as z
	LEFT JOIN CAR_PARK_FACILITY_TABLE as fa ON z.facility_id = fa.facility_id
	WHERE fa.FACILITY_NAME = 'Bella Vista Station Car Park'
EMIT CHANGES;
    

### Aggregated Availability by Facility with Five-Seconds of windows

In [ ]:
### working with success - Availability by Facility 
CREATE TABLE Availability_By_Facility AS
  SELECT f.facility_id , f.facility_name,  f.spots, z.zone_occupancy_total, f.spots - sum(z.zone_occupancy_total) as avail
    FROM  CAR_PARK_ZONE_STREAM  as z
    LEFT JOIN CAR_PARK_FACILITY_TABLE as f
    ON f.facility_id = z.facility_id
    WINDOW TUMBLING (SIZE 5 SECONDS)
    GROUP BY f.facility_id,  f.facility_name,  f.spots, z.zone_occupancy_total
    HAVING SUM(z.zone_occupancy_total) > 1
    EMIT CHANGES;